<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **COVID Dashboard**

## 1\. Contexto

Este será um projeto em parceria com a ebac aonde temos dados da covid-19.

Iremos extrai-los, trata-los e o salvar corretamente para que possamos estar fazendo um dashboard com algumas informações da covid-19 no Brasil no ano de 2021.

Teremos dois modelo de dados, um sobre os casos registrados e um sobre a vacinação.

Este será o de vacinações:

## 2\. Pacotes e bibliotecas

In [ ]:
import pandas as pd

## 3\. Extração

In [ ]:
# Vacinação:
# Vamos processar os dados de vacinação da universidade de Oxford.
# • Extração: Os dados estão compilados em um único arquivo:

vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv' , sep=',', parse_dates=[3], infer_datetime_format=True)

# Vamos selecionar as colunas de interesse e as linhas referentes ao Brasil:

vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True )
vaccines = vaccines[[ 'location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date' ]]

# • Wrangling: Vamos manipular os dados para o dashboard. O foco é garantir uma boa granularidade e qualidade da base de dados:
print(vaccines.shape, '\n')
print(vaccines.info(), '\n')

# Vamos começar tratando os dados faltantes. A estratégia será a de preencher os buracos com o valor anterior válido mais próximo:
vaccines = vaccines.fillna(method='ffill')

# Vamos também filtrar a base de dados de acordo com a coluna date para garantir que ambas as bases de dados tratam do mesmo período de tempo.
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

# Agora, vamos alterar o nome das colunas:
vaccines = vaccines.rename(columns={'location':'country', 'total_vaccinations':'total',
                                    'people_vaccinated':'one_shot', 'people_fully_vaccinated':'two_shots', 'total_boosters': 'three_shots', } )

## 4\. Transformação

In [ ]:
# Vamos então computar novas colunas para enriquecer a base de dados.

# - Chaves temporais:
vaccines['month'] = vaccines['date'].apply( lambda date: date.strftime('%Y-%m') )
vaccines['year'] = vaccines['date'].apply( lambda date: date.strftime('%Y') )

# - Dados relativos:
vaccines['one_shot_perc'] = round( vaccines['one_shot'] / vaccines['population'], 4 )
vaccines['two_shots_perc'] = round( vaccines['two_shots'] / vaccines['population'], 4 )
vaccines['three_shots_perc'] = round( vaccines['three_shots'] / vaccines['population'], 4 )

# Vamos fazer o type casting das colunas:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

# Por fim, vamos reorganizar as colunas.
vaccines = vaccines[['date','country','population','total','one_shot',
                     'one_shot_perc','two_shots','two_shots_perc','three_shots', 'three_shots_perc', 'month', 'year']]

## 5\. Carregamento

In [ ]:
# • Carregamento: Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carregá-lo no Google Data Studio:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)
vaccines

## 6. Link do notebook e do google data studio:
No google colab com ambos os dados, tanto casos quanto vacinações:  
-https://colab.research.google.com/drive/1uNk3G9-ZpyDsZ-PP6xdUBu6nwzJywF4F?usp=sharing               

já no google data studio contém o dashboard dos dados:  
-https://lookerstudio.google.com/reporting/a4b9b68e-83df-42e6-8b36-a85a65999de2